In [ ]:
import pandas as pd
import os
sim_t = 0.99
t_g1_total_count = 
t_g1_daily_max_count = 
t_g2_total_count = 
t_g2_daily_max_count = 

In [ ]:
home = r"data/"

In [ ]:
tabular_data_p2 = pd.read_csv(home+'g1_tabular_data.csv')

In [ ]:
ground_truths = tabular_data_p2[(tabular_data_p2['total_count']>=t_g2_total_count) | (tabular_data_p2['daily_max_count']>=t_g2_daily_max_count)]
ground_truths['post_release_count'] = ground_truths['total_count']

In [ ]:
g1_g2_text_embed_similar_pairs_processed_dir = home+"/output/embed-similarity-pairs-processed/"  
similar_data = pd.read_json(g1_g2_text_embed_similar_pairs_processed_dir+"merged_data_0.99_updated.json")
similar_data.columns = ['Project1CrashType','Project2CrashType']

In [ ]:
g1_tabular_data = pd.read_csv(home+'g2_tabular_data_400.csv')

In [ ]:
g1_tabular_data = g1_tabular_data[['CrashType','total_count',"daily_max_count"]]

In [ ]:
viral_g1_tabular_data = g1_tabular_data[(g1_tabular_data['total_count']>=t_g1_total_count)|(g1_tabular_data['daily_max_count']>=t_g1_daily_max_count)]

In [ ]:
viral_g1_similar_data = similar_data.merge(viral_g1_tabular_data, left_on='Project2CrashType', right_on='CrashType')
viral_g1_similar_data

In [ ]:
occurrence_df_counts_daily = pd.read_csv(home+"occurrence_df_counts_daily.csv")
g2_release_date = 'xxxx-xx-xx'
occurrence_df_counts_daily = occurrence_df_counts_daily[occurrence_df_counts_daily['project_key']=='g2']
pre_release_occurrences = occurrence_df_counts_daily[occurrence_df_counts_daily['CreationDateObj']<g2_release_date].groupby(['CrashType'])[['ReportIds']].sum().reset_index()
pre_release_counts = pre_release_occurrences.copy()
pre_release_counts.columns = ['CrashType','count']

In [ ]:
sim_only_suggestions = viral_g1_similar_data['Project1CrashType'].unique()
len(sim_only_suggestions)

In [ ]:
len(ground_truths[ground_truths['CrashType'].map(lambda x: x in sim_only_suggestions)]), ground_truths[ground_truths['CrashType'].map(lambda x: x in sim_only_suggestions)]['post_release_count'].sum()

In [ ]:
tabular_data_p2 = pd.read_csv(home+'g1_tabular_data.csv')
tabular_data_p2 = tabular_data_p2[['CrashType', 'total_count', 'daily_max_count']]
suggestions = []

# Define the folder path
g2_suggestions_dir = home+"scores-imbalance-platform-exception-400-components/"

# Loop through all files in the folder
for filename in os.listdir(g2_suggestions_dir):
    try:

        file_path = g2_suggestions_dir + filename 
        if '-False-' not in file_path: 
            print(filename)
            df = pd.read_json(file_path)#[['suggestions']]
            
            print(df['precision'])
            df = df[df['recall'].map(lambda x: x>0)&df['precision'].map(lambda x: x>0)]
            for loc in df.loc:
                try:
                    print(loc['suggestions'].values())
                    col = '-'.join(filename.split('.json')[0].split('-')[-2:])+'-'+loc['data']+'|g2|'
                    tabular_data_p2[col+'predictions'] = tabular_data_p2['CrashType'].map(lambda x: x in loc['suggestions'].values())
                    print("done")
                except Exception as e:
                    print("here1", e)
                    pass
    except Exception as e:
        print("here2", e)
        pass



In [ ]:
d = pd.read_json(home+"1.2 Get-similar-CrashType-data-v2-0.99-similar-groups.json")
d['data_similar_group'] = d['g2_CrashTypes']
g2_similar_groups =  d[['data_similar_group']]

dfs = []
for ground_truth in ground_truths['CrashType'].values:
    dfs.append(g2_similar_groups[g2_similar_groups['data_similar_group'].map(lambda li: ground_truth in li)])
df = pd.concat(dfs)
all_g2_ground_truth_CrashTypes = [k for CrashTypes in df['data_similar_group'].values.tolist() for k in CrashTypes]
all_g2_ground_truth_CrashTypes = list(set(all_g2_ground_truth_CrashTypes+ground_truths['CrashType'].values.tolist()))
all_g2_ground_truth_CrashTypes = pd. DataFrame({"CrashType":all_g2_ground_truth_CrashTypes})

In [ ]:
tabular_data_p2.head()

In [ ]:
from collections import Counter

In [ ]:
tabular_data_p2['ml_pred_aggregated'] = tabular_data_p2[[c for c in tabular_data_p2.columns if 'predictions' in c]].apply(lambda x: Counter(x.values).most_common(1)[0][0], axis=1)
tabular_data_p2['sim_pred_aggregated'] = tabular_data_p2['CrashType'].map(lambda x: x in sim_only_suggestions)
tabular_data_p2['ml_plus_sim_pred_aggregated'] = tabular_data_p2['ml_pred_aggregated']&tabular_data_p2['sim_pred_aggregated']
tabular_data_p2['ml_or_sim_pred_aggregated'] = tabular_data_p2['ml_pred_aggregated']|tabular_data_p2['sim_pred_aggregated']
ground_truths_CrashTypes  = ground_truths['CrashType'].values
tabular_data_p2['y_true']  = tabular_data_p2['CrashType'].map(lambda x: x in ground_truths_CrashTypes)
tabular_data_p2['y_relaxed_true']  = tabular_data_p2['CrashType'].map(lambda x: x in all_g2_ground_truth_CrashTypes['CrashType'].unique())

In [ ]:
g2_similar_groups['group_number'] = [i for i in range(len(g2_similar_groups))]

In [ ]:
g2_similar_groups_explode = g2_similar_groups.explode('data_similar_group')
g2_similar_groups_explode.columns = ['CrashType', 'similar_group']

In [ ]:
tabular_data_p2 = tabular_data_p2.merge(g2_similar_groups_explode, on="CrashType", how="left")

In [ ]:
tabular_data_p2['processed_similar_group'] = tabular_data_p2.apply(lambda r: r['CrashType'] if str(r['similar_group'])=='nan' else r['similar_group'], axis=1)

In [ ]:
pre_release_counts = pre_release_counts.merge(g2_similar_groups_explode, on="CrashType", how="left")
pre_release_counts['processed_similar_group'] = pre_release_counts.apply(lambda r: r['CrashType'] if str(r['similar_group'])=='nan' else r['similar_group'], axis=1)

In [ ]:
tabular_data_p2_ = tabular_data_p2.copy()
tabular_data_p2_['post_release_count'] = tabular_data_p2_['total_count']

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score, recall_score, matthews_corrcoef

ml_li = []
ml_relaxed_li = []

auc_baseline = 0.5
auprc_baseline = len(tabular_data_p2[tabular_data_p2['y_true']])/len(tabular_data_p2)

for count_t in range(0,10):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'ml_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                  "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })


sim_li = []
sim_relaxed_li = []

for count_t in range(0,10):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    sim_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                   "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })
    


ml_sim_li = []
ml_sim_relaxed_li = []

for count_t in range(0,10):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'ml_plus_sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")

    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_sim_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
        "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })
    
ml_or_sim_li = []
ml_or_sim_relaxed_li = []

for count_t in range(0,10):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'ml_or_sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")

    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_or_sim_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
             "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })
    
    y_true_col = 'y_relaxed_true'
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_or_sim_relaxed_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })

In [ ]:
import matplotlib.pyplot as plt
# Plot the data

metrics = {
"precision": "Precision",
 "recall":"Recall",
 "auc":"AUROC",
    "auprc": "AUPRC",
    "mcc":"MCC"
}

for metric in metrics.keys():
    print(metric)
    
    plt.figure(figsize=(5, 2))
    plt.plot([o[metric] for o in ml_li], color = 'black', label='ML - '+metrics[metric], linestyle = '-.')
    
    plt.plot([o[metric] for o in sim_li], color = 'black', label='Similarity - '+metrics[metric], linestyle = '--')
    
    plt.plot([o[metric] for o in ml_sim_li], color = 'black', label='ML & Similarity - '+metrics[metric], linestyle = '-')

    if metric =='auc':
        plt.axhline(y=auc_baseline, color='black', linestyle='dotted', label='Baseline '+metrics[metric])
    elif metric =='auprc':
        plt.axhline(y=auprc_baseline, color='black', linestyle='dotted', label='Baseline '+metrics[metric])
    
    
    
    # Rotate x labels
    # plt.xticks(rotation=45)
    #plt.legend(bbox_to_anchor=(0.9, -0.5))
    
    # Add labels and title
    plt.xlabel('Pre-release Count Threshold')
    plt.ylabel(metrics[metric])

    
    # Show plot
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    plt.savefig('g1-g2-recall.pdf', bbox_inches='tight')  # Save with tight layout


In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score, recall_score, matthews_corrcoef

ml_li = []
ml_relaxed_li = []

auc_baseline = 0.5
auprc_baseline = len(tabular_data_p2[tabular_data_p2['y_true']])/len(tabular_data_p2)

for count_t in range(0,1):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'ml_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })


sim_li = []
sim_relaxed_li = []

for count_t in range(0,1):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    sim_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })
    


ml_sim_li = []
ml_sim_relaxed_li = []

for count_t in range(0,1):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'ml_plus_sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")

    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_sim_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })
    
ml_or_sim_li = []
ml_or_sim_relaxed_li = []

for count_t in range(0,1):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'ml_or_sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")

    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_or_sim_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })
    
    y_true_col = 'y_relaxed_true'
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index().merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index().merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    ml_or_sim_relaxed_li.append({"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              })

In [ ]:
d = pd. DataFrame([ml_li[0],sim_li[0], ml_sim_li[0],ml_or_sim_li[0]])
d["method"] = ["ml_li", "sim_li", "ml_sim_li","ml_or_sim_li"]

In [ ]:
d_ = d[['method', 'precision', 'recall', 'accuracy', 'auc', 'auprc', 'mcc', 'TPs_occurrences', 'FPs_occurrences']]
d_

In [ ]:
for count_t in range(0,1):

    pre_release_important_crashes = pre_release_counts[pre_release_counts['count']>count_t]['CrashType'].values


    
    y_true_col = 'y_true'
    y_pred_col = 'sim_pred_aggregated'
    
    tabular_data_p2_ = tabular_data_p2.copy()
    tabular_data_p2_['important'] = tabular_data_p2_['CrashType'].map(lambda x: x in pre_release_important_crashes)
    tabular_data_p2_[y_pred_col] = tabular_data_p2_[y_pred_col] & tabular_data_p2_['important']
    print(y_pred_col,len(tabular_data_p2_[tabular_data_p2_[y_pred_col]]), 'suggestions', len(tabular_data_p2_[tabular_data_p2_[y_pred_col]])/len(tabular_data_p2_)*100, "%")
    auc = roc_auc_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    auprc = average_precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    precision = precision_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    recall = recall_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    accuracy = accuracy_score(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    mcc = matthews_corrcoef(tabular_data_p2_[y_true_col], tabular_data_p2_[y_pred_col])
    TPs = tabular_data_p2_[tabular_data_p2_[y_true_col] & tabular_data_p2_[y_pred_col]]
    FPs = tabular_data_p2_[(tabular_data_p2_[y_true_col]==False) & tabular_data_p2_[y_pred_col]]
    FNs = tabular_data_p2_[(tabular_data_p2_[y_true_col]) & (tabular_data_p2_[y_pred_col]==False)]
    TPs_occurrences = TPs.reset_index(#['CrashType'].map(lambda x: str(int(x)) if type(x)==type(0.0) else x).reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    FPs_occurrences = FPs.reset_index(#).merge(g2_similar_groups_explode, on="processed_similar_group"
    ).merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()
    
    sim_li = {"auc":auc,
               "auprc":auprc, 
               "precision":precision,
               "recall":recall,
                   "accuracy": accuracy,
               "mcc":mcc,
               "TPs":TPs,
               "FPs":FPs,
               "FNs":FNs,
               "TPs_occurrences": TPs_occurrences,
               "FPs_occurrences": FPs_occurrences
              }

In [ ]:
tabular_data_p2_[tabular_data_p2_['y_true']].merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()

In [ ]:
tabular_data_p2_.merge(tabular_data_p2_, on="CrashType")['post_release_count'].sum()

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(sim_li['FNs']['total_count'], sim_li['TPs']['total_count'], method="exact", alternative='less')
print(U1,p)

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(sim_li['FNs']['daily_max_count'], sim_li['TPs']['daily_max_count'], method="exact", alternative='less')
print(U1,p)

In [ ]:
df = pd.DataFrame({"count": sim_li['FNs']['daily_max_count'].values,
                   "Type_of_count":["Daily max count" for i in range(len(sim_li['FNs']))],
                   "":["FNs" for i in range(len(sim_li['FNs']))]
                  })._append(pd.DataFrame({"count": sim_li['FNs']['total_count'].values,
                   "Type_of_count":["Total count" for i in range(len(sim_li['FNs']))],
                   "":["FNs" for i in range(len(sim_li['FNs']))]
                  }))._append(pd.DataFrame({"count": sim_li['TPs']['daily_max_count'].values,
                   "Type_of_count":["Daily max count" for i in range(len(sim_li['TPs']))],
                   "":["TPs" for i in range(len(sim_li['TPs']))]
                  })
                  )._append(pd.DataFrame({"count": sim_li['TPs']['total_count'].values,
                   "Type_of_count":["Total count" for i in range(len(sim_li['TPs']))],
                   "":["TPs" for i in range(len(sim_li['TPs']))]
                  }))

df

In [ ]:
from sklearn import preprocessing
df_total_count = df[df['Type_of_count']=='Total count']
normalized_arr = preprocessing.normalize([df_total_count['count']])
df_total_count['normalized_count'] = normalized_arr[0]
print(df_total_count)

df_daily_max_count = df[df['Type_of_count']=='Daily max count']
normalized_arr = preprocessing.normalize([df_daily_max_count['count']])
df_daily_max_count['normalized_count'] = normalized_arr[0]

normalized_df = df_total_count._append(df_daily_max_count)

In [ ]:
normalized_df.columns = ['count', 'Type_of_count', '', 'Normalized count']

In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.ticker as ticker


plt.figure(figsize=(5, 3))
ax = sns.violinplot(data=normalized_df, y="Type_of_count", x="Normalized count", hue="", split=True, inner="quart", scale='count', palette=["lightgrey", "darkgrey"])
plt.xscale('log')
i=0
for l in ax.lines:
    if i in [1,4,7,10]:
        l.set_linestyle('-')
    else:
        l.set_linestyle('--')
    l.set_color('black')
    l.set_alpha(0.8)
    i=i+1
ax.set_ylabel('')
plt.savefig('nomalized_bean_plot.pdf', bbox_inches='tight')

In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.ticker as ticker


plt.figure(figsize=(5, 2))
ax = sns.violinplot(data=normalized_df, y="Type_of_count", x="Normalized count", hue="", split=True, inner="quart", scale='count', palette=["lightgrey", "darkgrey"])
plt.xscale('log')
i=0
for l in ax.lines:
    if i in [1,4,7,10]:
        l.set_linestyle('-')
    else:
        l.set_linestyle('--')
    l.set_color('black')
    l.set_alpha(0.8)
    i=i+1
ax.set_ylabel('')
plt.savefig('nomalized_bean_plot.pdf', bbox_inches='tight')

In [ ]:
from cliffs_delta import cliffs_delta
d, res = cliffs_delta(df_total_count[df_total_count[""]=='FNs']['normalized_count'].values, 
                      df_total_count[df_total_count[""]=='TPs']['normalized_count'].values
                      
)

print(d,res)

In [ ]:
from cliffs_delta import cliffs_delta
d, res = cliffs_delta(df_daily_max_count[df_daily_max_count[""]=='FNs']['normalized_count'].values, 
                      df_daily_max_count[df_daily_max_count[""]=='TPs']['normalized_count'].values
                      
)

print(d,res)                      
                  